## Import

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
import pandas as pd
import numpy as np
import xgboost as xgb
import matplotlib.pyplot as plt
import lightgbm as lgb
from sklearn.metrics import r2_score, mean_squared_error
import datetime
from numpy import array, linspace
from xgboost import XGBClassifier
from matplotlib.pyplot import plot
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from fbprophet import Prophet

In [ ]:
from astral.sun import sun
from astral import LocationInfo

# Data Preparation

---

In [ ]:
def timeofday(x):
    if (x == 23) or (x <5):
        return 'night'
    elif (x>=5) and (x<9):
        return 'early_morning'
    elif (x>=9) and (x<15):
        return 'morning'
    elif (x>=15) and (x<20):
        return 'afternoon'
    elif (x>=20) and (x<23):
        return 'evening'

In [ ]:
def X_prep(data):
    
    data = data.drop(columns = [data.columns[-1]])
    
    data["time_step"] = pd.to_datetime(data["time_step"], format="%Y-%m-%dT%H:%M:%S.%f")
    
    # half_hour
    data["half_h"] = data['time_step'].dt.floor('30T').dt.time
    data["half_hour"] = data["half_h"].apply(lambda x: str(x)[:5])
    data = data.drop('half_h', axis = 1)

    # hour, weekday, week, month
    data['hour'] = data.time_step.dt.hour
    data['weekday'] = data.time_step.dt.weekday
    data['week'] = data.time_step.dt.week
    data['month'] = data.time_step.dt.month
    
    # season
    seasons = ['winter', 'winter', 'spring', 'spring', 'spring', 'summer', 'summer', 'summer', 
               'fall', 'fall', 'fall', 'winter']
    month_to_season = dict(zip(range(1,13), seasons))
    data['season'] = data.month.map(month_to_season)

    # timeofday
    data['timeofday'] = data.hour.apply(lambda x: timeofday(x))
    
    # daylight time inversed
    city = LocationInfo("Paris")
    _ = data.time_step.apply(lambda x: sun(city.observer, date=x.date()))
    data["daylight_inv"] = _.apply(lambda x: 1/((x["sunset"] - x["sunrise"]).seconds/3600))
    
    # dealing with Consumption missing values
    data['consumption'].interpolate(method='linear', inplace=True)
    
    # dealing with weather missing values
    data["datetime"] = data["time_step"].apply(lambda x: "{:%m, %d, %H}".format(x))
    data.iloc[0,2:9] = data.iloc[59,2:9]
    _ = data[["datetime","visibility","temperature","humidity","humidex","windchill","wind","pressure"]] 
    info = _.drop_duplicates(subset ="datetime", keep = "first").reset_index(drop=True)
    _ = info.drop(columns = ['datetime'])
    _.interpolate(method='linear', inplace=True)
    info.iloc[:,1:] = _
    data.iloc[:,2:9] = pd.merge(info,data.datetime, on="datetime").iloc[:,1:8]
  
    # drop time_step, datetime
    data = data.drop(["datetime"], axis = 1)
    data.set_index("time_step", inplace=True)

    return data

In [ ]:
X_train = Xtrain.copy()
X_train = X_prep(X_train)

In [ ]:
def y_prep(y_train):
    y_train.time_step = pd.to_datetime(y_train.time_step)
    y_train.set_index("time_step", inplace=True)
    y_train.interpolate(method='linear', inplace=True)
    return y_train

In [ ]:
y_train = ytrain.copy()
y_train = y_prep(y_train)

# Data Exploration


---




In [ ]:
data_explor = pd.concat([X_train,
                         y_train[['washing_machine','fridge_freezer','TV','kettle']]], axis = 1)

### General exploratory analysis

Before looking at our data_explor dataframe, let's check the state of our target variables before the transformations we applied.

In [ ]:
ytrain.columns

In [ ]:
ytrain.shape

In [ ]:
ytrain.isnull().sum()

We are dealing with 4 target variables and we have 417,599 observations in our training dataset. Among these observations, 10,231 values are null (or 2.4% of the training data).

After having applied a linear interpolation to fill the null values, here are some basic information on the target variables:

In [ ]:
data_explor[["washing_machine", "fridge_freezer", "TV", "kettle"]].describe()

- **Washing Machine**: the standard deviation is 10 times the mean, which warns us about the repartition of our data. Furthermore, the 3rd quartile is 0, meaning that at least 75% of the data is zero. Moreover, the maximum value is 486 the mean value. We may have extreme values.

- **Fridge Freezer**: Here, the standard deviation is around the same value as the mean. At least 25% of the data is zero, which is sparse.

- **TV**: Regarding TV, all the values seem to stay around 7. The repartition looks more uniform.

- **Kettle**: We seem to be in the same situation as that of Washing Machine. There are some extreme values and the data is very sparse.

Let us first explore the repartition of our target variables with histograms.

In [ ]:
# Histograms
fig, ax = plt.subplots(1,4, figsize=(18,2))
ax[0].hist(data_explor.washing_machine)
ax[0].set_title("Washing machine")

ax[1].hist(data_explor.fridge_freezer)
ax[1].set_title("Fridge Freezer")

ax[2].hist(data_explor.TV)
ax[2].set_title("TV")

ax[3].hist(data_explor.kettle)
ax[3].set_title("Kettle")

In any case, we are dealing with very sparse variables and none of them is normally distributed. 

Washing Machine and Kettle seem to have very few and extremely large values. Let us observe the repartition of the extreme values for these two variables.  

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(18,2))
ax[0].hist(data_explor.washing_machine[data_explor.washing_machine > 100])
ax[0].set_title("Washing machine")

ax[1].hist(data_explor.kettle[data_explor.kettle > 100])
ax[1].set_title("Kettle")

Now, what about the correlations between all available variables ? 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
cor = data_explor.corr()
plt.figure(figsize=(4,4))
sns.heatmap(cor)

- The variables seem fairly correlated to Consumption, especially Kettle
- TV is correlated to the hour of the day
- The environmental informations (humidity, temperature...) are quite correlated 
- However, there is low correlation between our target variables and the environmental information
- Our target variables are not correlated with one another, ruling out the possibility of using the predictions of one variable to predict another variable.

Let us observe in more details the relation between consumption and our target variables.

In [ ]:
fig, ax = plt.subplots(1,4, figsize=(18,4))

for i in range(4):
    mask = np.logical_and(data_explor.index.month == 5, data_explor.index.day==11 + i)
    ax[i].plot(data_explor[mask].washing_machine, label="washing_machine")
    ax[i].plot(data_explor[mask].consumption, label="consumption")
    ax[i].set_title("Month 5, day " + str(11+i))
    ax[i].legend()
    ax[i].axis('off')

We can notice a specific pattern in consumption when the washing machine is in use. We first have a large increase and then many ups and downs. 

In [ ]:
fig, ax = plt.subplots(1,4, figsize=(18,4))

for i in range(4):
    mask = np.logical_and(data_explor.index.month == 5, data_explor.index.day==11 + i)
    ax[i].plot(data_explor[mask].fridge_freezer, label="fridge_freezer")
    ax[i].plot(data_explor[mask].consumption, label="consumption")
    ax[i].set_title("Month 5, day " + str(11+i))
    ax[i].legend()
    ax[i].axis('off')

It seems that consumption reproduces the behavior of that of the fridge_freezer, but not always. That's why there is also a small correlation between the two variables. 

In [ ]:
fig, ax = plt.subplots(1,4, figsize=(18,4))

for i in range(4):
    mask = np.logical_and(data_explor.index.month == 5, data_explor.index.day==11 + i)
    ax[i].plot(data_explor[mask].TV, label="TV")
    ax[i].plot(data_explor[mask].consumption, label="consumption")
    ax[i].set_title("Month 5, day " + str(11+i))
    ax[i].legend()
    ax[i].axis('off')

TV does not seem to influence on the overall consumption at all. 

In [ ]:
fig, ax = plt.subplots(1,4, figsize=(18,4))

for i in range(4):
    mask = np.logical_and(data_explor.index.month == 4, data_explor.index.day==11 + i)
    ax[i].plot(data_explor[mask].kettle, label="Kettle")
    ax[i].plot(data_explor[mask].consumption, label="consumption")
    ax[i].set_title("Month 5, day " + str(11+i))
    ax[i].legend()
    ax[i].axis('off')

Whenever the kettle is turned on, we observe a peak in the overall consumption. It's a very sharp peak which seems to last only for a few minutes. It could be very interesting to spot that peak in our data in order to find the kettle consumption peak. 

### By month

In [ ]:
by_month = data_explor[['consumption','washing_machine','fridge_freezer',
                        'TV','kettle','month']].groupby('month').mean()
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(15,2))

by_month.washing_machine.plot.bar(ax=axes[0],title='Washing Machine')
by_month.fridge_freezer.plot.bar(ax=axes[1],title='Fridge Freezer')
by_month.TV.plot.bar(ax=axes[2],title='TV')
by_month.kettle.plot.bar(ax=axes[3],title='Kettle')

We can make the following observations:
* **Kettle:** it is more used during fall and winter (starting October) than during warmer months
* It is difficult to observe a significant correlation between months and the consumption of fridge, washing machine and TV



### By season

In [ ]:
by_hour = data_explor[['consumption','washing_machine','fridge_freezer',
                       'TV','kettle','hour','season']]
by_hour_spring = by_hour[by_hour.season == "spring"].groupby(['hour']).mean()
by_hour_summer = by_hour[by_hour.season == "summer"].groupby(['hour']).mean()
by_hour_winter = by_hour[by_hour.season == "winter"].groupby(['hour']).mean()
by_hour_fall = by_hour[by_hour.season == "fall"].groupby(['hour']).mean()

fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(18,4))

by_hour_spring.washing_machine.plot(ax=axes[0],title='Washing Machine')
by_hour_summer.washing_machine.plot(ax=axes[0])
by_hour_winter.washing_machine.plot(ax=axes[0])
by_hour_fall.washing_machine.plot(ax=axes[0])

by_hour_spring.fridge_freezer.plot(ax=axes[1],title='Fridge Freezer')
by_hour_summer.fridge_freezer.plot(ax=axes[1])
by_hour_winter.fridge_freezer.plot(ax=axes[1])
by_hour_fall.fridge_freezer.plot(ax=axes[1])

by_hour_spring.TV.plot(ax=axes[2],title='TV')
by_hour_summer.TV.plot(ax=axes[2])
by_hour_winter.TV.plot(ax=axes[2])
by_hour_fall.TV.plot(ax=axes[2])

by_hour_spring.kettle.plot(ax=axes[3],title='Kettle')
by_hour_summer.kettle.plot(ax=axes[3])
by_hour_winter.kettle.plot(ax=axes[3])
by_hour_fall.kettle.plot(ax=axes[3])

axes[0].legend(['spring','summer','winter','fall'])
axes[1].legend(['spring','summer','winter','fall'])
axes[2].legend(['spring','summer','winter','fall'])
axes[3].legend(['spring','summer','winter','fall'])

- **Fridge_freezer**: consumption seems to be higher in summer and fall than in spring and winter
- **TV**: consumption is pretty homogenous among season. TV is more used in the morning and early afternoon during spring. During winter it is used later at night.
- **Kettle**: consumption is significantly higher during winter and fall
- **Washing machine**: consumption seems homogenous among seasons

### By weekday

In [ ]:
by_weekday = data_explor[['consumption','washing_machine','fridge_freezer',
                          'TV','kettle','weekday']].groupby('weekday').mean()
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(15,2))

by_weekday.washing_machine.plot.bar(ax=axes[0],title='Washing Machine')
by_weekday.fridge_freezer.plot.bar(ax=axes[1],title='Fridge Freezer')
by_weekday.TV.plot.bar(ax=axes[2],title='TV')
by_weekday.kettle.plot.bar(ax=axes[3],title='Kettle')

- Weekday doesn't seem to have an importance for the consumption of fridge, TV and Kettle
- Washing machine is more used during the weekend, notably on Sunday

### By time of day

In [ ]:
by_timeofday = data_explor[['consumption','washing_machine','fridge_freezer',
                            'TV','kettle','timeofday']].groupby('timeofday').mean()
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(15,2))

by_timeofday.washing_machine.plot.bar(ax=axes[0],title='Washing Machine')
by_timeofday.fridge_freezer.plot.bar(ax=axes[1],title='Fridge Freezer')
by_timeofday.TV.plot.bar(ax=axes[2],title='TV')
by_timeofday.kettle.plot.bar(ax=axes[3],title='Kettle')                      

This is how we defined the times of day: 5am-9am is **early_morning**, 9am-3pm is **morning**, 3pm-8pm **afternoon**, 8pm-11pm is **evening**, 11pm-5am is **night**
- **Fridge**: consumption is homogeneous during the day
- **TV**: consumption is higher during afternoon and evening
- **Kettle**: it consumes the most in the afternoon
- **Washing machine**: consumption is higher during evening and night

# Final Models


---



## Fridge Freezer

**Pattern** \
Thanks to the Exploratory Data Analysis, we are able to notice that the pattern in the fridge_freezer consumption is often reflected on the pattern of the overall consumption. However, if the overall consumption is above a certain level, the pattern of the fridge freezer consumption is no longer reflected on the overall consumption. 

Therefore, we put in our training data the overall consumption over the past 30 minutes to caracterize the pattern; however, if the consumption is higher than 600, we leave that value at 600: it is capped. 

**Weather Information** \
It seems that this information influence the fridge_freezer consumption, so we keep it. This makes sense since, for example, the higher the outdoor temperature, the higher the indoor temperature and so the more energy the fridge freezer needs to keep its inside temperature low. 

**Temporal Information** \
Week, weekday and hour of the day are taken into account. 

**Model** \
We use a boosting method: the Light GBM algorithm. 


In [ ]:
class fridge_freezer():
    def __init__(self):
        self.model = lgb.LGBMRegressor(n_estimators=1700, min_child_weight=1.5, max_depth=12, 
                               max_bin=100, colsample_bytree=0.6)
    
    
    def shift_data(self, data, lag=30):
        series = [data.shift(i) for i in range(lag-1, -1, -1)]
        new_df = pd.concat(series, axis=1)
        name_col = []
        
        for i in range(lag, 0, -1):
            var = "consumption -" + str(i)
            name_col.append(var)
        
        new_df.columns = name_col
        new_df["std"] = new_df.std(axis=1)
        new_df["mean"] = new_df.mean(axis=1)
        return new_df
    
    def transform(self, X_train):
        X_train = X_train.drop(columns=["Unnamed: 9"])
        X_train.time_step = pd.to_datetime(X_train.time_step)
        X_train["week"] = X_train["time_step"].map(lambda x: x.week)
        X_train["weekday"] = X_train["time_step"].map(lambda x: x.weekday)
        X_train["hour"] = X_train["time_step"].map(lambda x: x.hour)
        
        #Missing values
        X_train.fillna(method="ffill", inplace=True) #Get the previous non null values
        
        #Conditions on consumption
        X_train["capped_consumption"] = np.where(X_train.consumption>=600, 600, X_train.consumption)
        
        #Supervised consumption
        supervised_consumption = self.shift_data(X_train.consumption)
        X_train = X_train.join(supervised_consumption)
        
        #One hot encoding
        X_train_hour = pd.Categorical(X_train.hour, categories = [i for i in range(24)])
        X_train_weekday = pd.Categorical(X_train.weekday, categories = [i for i in range(7)])
        X_train_week = pd.Categorical(X_train.week, categories = [i for i in range(52)])
        
        X_train = X_train.join(pd.get_dummies(X_train_hour, prefix="hour"))
        X_train = X_train.join(pd.get_dummies(X_train_weekday, prefix="weekday"))
        X_train = X_train.join(pd.get_dummies(X_train_week, prefix="week"))
        
        X_train.drop(columns=["time_step", "hour", "weekday", "week"], inplace=True)
        return X_train
    
    def fit(self, X_train, y_train):
        self.model.fit(X_train, y_train)
        
    def predict(self, X_test):
        y_pred = self.model.predict(X_test)
        y_pred = np.where(y_pred<0, 0, y_pred)
        return y_pred

In [ ]:
prep = fridge_freezer()
X_train = prep.transform(Xtrain)

In [ ]:
# Drop na
data = X_train.join(ytrain)
data = data.dropna()
X_train = data.iloc[:,:-5]
y_train = data.iloc[:,-5:]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, shuffle=False)

In [ ]:
# Fitting of the model
prep.fit(X_train, y_train.fridge_freezer)

# Predict
y_pred = prep.predict(X_test)

In [ ]:
print("R2 of", r2_score(y_test.fridge_freezer, y_pred))
print("RMSE of", np.sqrt(mean_squared_error(y_test.fridge_freezer, y_pred)))
print("Adjusted RMSE:", np.sqrt(mean_squared_error(y_test.fridge_freezer, y_pred)*49.79/74.86))

## Kettle

As seen in the exploratory data anaylsis, the overall consumption is very much correlated with the Kettle consumption. More specifically, we notice that, usually, when the overall consumption is below a certain level, the kettle is not turned on. 

Therefore, we decided to create **two models**: 
- One model when the overall consumption was below 600
- Another model when the overall consumption was above 600 (and the values to predict were likely to go to extremes)

**Type of models** \
Boosting was the chosen method for this regression problem. More specifically, we selected the XGBoost algorithm.

**Peak** \
Since a peak in consumption is likely to equal a peak in the kettle consumption, we wanted to caracterize the peak in the overall consumption in our training data. To do so, we decided to take the overall consumption over the last 20 minutes, their mean and standard deviation. Thanks to this data we can say that, if the standard deviation is high and the mean is high, we are at the beginning of the peak: we should predict a high value for kettle. 

**Temporal Information** \
We chose to keep the following temporal features, as they seemed relevant in our exploratory data analysis: hour, weekday and week number. 

**Weather Information** \
We also dropped the environmental information (temperature, humidity...) as they didn't seem to have an impact on the kettle consumption. 

In [ ]:
class kettle():
    
    def __init__(self):
        self.model_0 = xgb.XGBRegressor(min_child_weight=1.5, max_depth= 8, gamma=0.6, colsample_bytree=0.9)
        self.model_1 = xgb.XGBRegressor(min_child_weight=1.5, max_depth= 8, gamma=0.6, colsample_bytree=0.9)
        
    def shift_data(self, data, lag=20):
        series = [data.shift(i) for i in range(lag-1, -1, -1)]
        new_df = pd.concat(series, axis=1)
        name_col = []
        
        for i in range(lag, 0, -1):
            var = "consumption -" + str(i)
            name_col.append(var)
        
        new_df.columns = name_col
        new_df["std"] = new_df.std(axis=1)
        new_df["mean"] = new_df.mean(axis=1)
        
        return new_df
    
    def transform(self, X_train):
        X_train = X_train[["time_step", "consumption"]]
        X_train.time_step = pd.to_datetime(X_train.time_step)
        
        X_train["hour"] = X_train["time_step"].map(lambda x: x.hour)
        X_train["weekday"] = X_train["time_step"].map(lambda x: x.dayofweek)
        X_train["week"] = X_train["time_step"].map(lambda x: x.week)
        
        #More information on consumption
        X_train["cluster"] = np.where(X_train.consumption > 600, 1, 0)
        
        #Missing values
        X_train.fillna(method="ffill", inplace=True) #Get the previous non null values
        
        #Supervised consumption
        supervised_consumption = self.shift_data(X_train.consumption)
        
        #One hot encoding
        X_train_hour = pd.Categorical(X_train.hour, categories = [i for i in range(24)])
        X_train_weekday = pd.Categorical(X_train.weekday, categories = [i for i in range(7)])
        X_train_week = pd.Categorical(X_train.week, categories = [i for i in range(52)])

        X_train = X_train.join(pd.get_dummies(X_train_hour, prefix="hour"))
        X_train = X_train.join(pd.get_dummies(X_train_weekday, prefix="weekday"))
        X_train = X_train.join(pd.get_dummies(X_train_week, prefix="week"))
        
        X_train.drop(columns=["time_step","hour", "weekday", "week"], inplace=True)
        
        return X_train.join(supervised_consumption)
    
    def fit(self, X_train, y_train):
        X_train_0 = X_train[X_train.cluster == 0]
        X_train_1 = X_train[X_train.cluster == 1]

        y_train_0 = y_train[X_train.cluster == 0]
        y_train_1 = y_train[X_train.cluster == 1]
        
        self.model_0.fit(X_train_0, y_train_0)
        self.model_1.fit(X_train_1, y_train_1)
        
    def predict(self, X_test):
        y_pred_kettle_0 = self.model_0.predict(X_test[X_test.cluster==0])
        y_pred_kettle_1 = self.model_1.predict(X_test[X_test.cluster==1])
        
        df_0 = pd.DataFrame(y_pred_kettle_0, index=X_test[X_test.cluster == 0].index)
        df_1 = pd.DataFrame(y_pred_kettle_1, index=X_test[X_test.cluster == 1].index)
        y_pred_kettle = pd.concat([df_0, df_1])
        y_pred_kettle.sort_index(inplace=True)
        
        y_pred_kettle = np.where(y_pred_kettle<0, 0, y_pred_kettle)
        
        return y_pred_kettle

In [ ]:
# Transform the data
prep = kettle()
X_train = prep.transform(Xtrain)

In [ ]:
# Drop na
data = X_train.join(ytrain)
data = data.dropna()
X_train = data.iloc[:,:-5]
y_train = data.iloc[:,-5:]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, shuffle=False)

In [ ]:
# Fitting of the model
prep.fit(X_train, y_train.kettle)

# Predict
y_pred = prep.predict(X_test)

In [ ]:
print("R2 of", r2_score(y_test.kettle, y_pred))
print("RMSE of", np.sqrt(mean_squared_error(y_test.kettle, y_pred)))
print("RMSE of", np.sqrt(mean_squared_error(y_test.kettle, y_pred))*4.95/74.86)

*Remark: the chosen hyperparameters are the results of a RandomizedSearchCV (sklearn)*

## Washing_machine

**Pattern** \
Thanks to the Exploratory Data Analysis, we are able to notice that the pattern in the washing_machine consumption is often reflected on the pattern of the overall consumption. Therefore, we put in our training data the overall consumption over the past 30 minutes to caracterize the pattern. 
Thanks to this data we can say that, if the standard deviation is high and the mean is high, we are at the beginning of the peak: we should predict a high value for washing_machine.

**Weather Information** \
We kept this information because it seems to have an influence on the washing_machine consumption. 

**Temporal Information** \
Week, weekday and hour of the day are taken into account. 

**Model** \
We use a boosting method: the Light GBM algorithm. 

In [ ]:
class washing_machine():
    
    def __init__(self):
        self.model = 0
        
    def shift_data(self, data, lag=30):
        series = [data.shift(i) for i in range(lag-1, -1, -1)]
        new_df = pd.concat(series, axis=1)
        name_col = []
        
        for i in range(lag, 0, -1):
            var = "consumption -" + str(i)
            name_col.append(var)
        
        new_df.columns = name_col
        new_df["std"] = new_df.std(axis=1)
        new_df["mean"] = new_df.mean(axis=1)
        
        return new_df
    
    def transform(self, X_train):
        
        #Environment information
        X_train = X_train.drop(columns="Unnamed: 9")
        X_train.time_step = pd.to_datetime(X_train.time_step)
        X_train["week"] = X_train["time_step"].map(lambda x: x.week)
        X_train["weekday"] = X_train["time_step"].map(lambda x: x.weekday)
        X_train["hour"] = X_train["time_step"].map(lambda x: x.hour)
        
        #Missing values
        X_train.fillna(method="ffill", inplace=True) #Get the previous non null values
        
        #Supervised consumption
        supervised_consumption = self.shift_data(X_train.consumption)
        X_train = X_train.join(supervised_consumption)
        
        #One hot encoding
        X_train_hour = pd.Categorical(X_train.hour, categories = [i for i in range(24)])
        X_train_weekday = pd.Categorical(X_train.weekday, categories = [i for i in range(7)])
        X_train_week = pd.Categorical(X_train.week, categories = [i for i in range(52)])
        
        X_train = X_train.join(pd.get_dummies(X_train_hour, prefix="hour"))
        X_train = X_train.join(pd.get_dummies(X_train_weekday, prefix="weekday"))
        X_train = X_train.join(pd.get_dummies(X_train_week, prefix="week"))
        
        X_train.drop(columns=["time_step", "hour", "weekday", "week"], inplace=True)
        
        return X_train
    
    def fit(self, X_train, y_train):
        lgb_train = lgb.Dataset(X_train.values, y_train)
        params = {
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': {'l2', 'l1'},
            'num_leaves': 50,
            'learning_rate': 0.05,
            'feature_fraction': 0.9,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'verbose': 0}

        self.model = lgb.train(params, lgb_train,
                             num_boost_round=10000)
        
    def predict(self, X_test):
        y_pred = self.model.predict(X_test, num_iteration=self.model.best_iteration)
        y_pred = np.where(y_pred<0,0,y_pred)
        return y_pred

In [ ]:
# Transform the data
prep = washing_machine()
X_train = prep.transform(Xtrain)

In [ ]:
# Drop na
data = X_train.join(ytrain)
data = data.dropna()
X_train = data.iloc[:,:-5]
y_train = data.iloc[:,-5:]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, shuffle=False)

In [ ]:
# Fitting of the model
prep.fit(X_train, y_train.washing_machine)

# Predict
y_pred = prep.predict(X_test)

In [ ]:
print("R2 of", r2_score(y_test.washing_machine, y_pred))
print("RMSE of", np.sqrt(mean_squared_error(y_test.washing_machine, y_pred)))
print("Adjusted RMSE: ", np.sqrt(mean_squared_error(y_test.washing_machine, y_pred))*5.55/74.86)

## TV

For TV, we used Prophet, open source software released by Facebook’s Core Data Science team. This method outperformed all our other trials using boosting algorithms.

Prophet is a procedure for forecasting time series data based on an additive model where non-linear trends are fit with yearly, weekly, and daily seasonality. It works best with time series that have strong seasonal effects and several seasons of historical data such as the consumption of TV.

In [ ]:
class tv():
    def __init__(self):
        self.model = Prophet()
    
    def transform(self, X_train):
        X_train.time_step = pd.to_datetime(X_train.time_step)
        df = pd.DataFrame({"ds": X_train.time_step})
        return df
    
    def fit(self, X_train, y_train):
        X_train['y'] = y_train
        self.model.fit(X_train)
        
    def predict(self, X_test):
        pred = self.model.predict(X_test)
        y_pred = np.where(pred.yhat <0, 0, pred.yhat)
        return y_pred

In [ ]:
# Transform the data
prep = tv()
X_train = prep.transform(Xtrain)

In [ ]:
# Drop na
data = X_train.join(ytrain)
data = data.dropna()
X_train = data.iloc[:,:-5]
y_train = data.iloc[:,-5:]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, shuffle=False)

In [ ]:
# Fitting of the model
prep.fit(X_train, y_train.TV)

# Predict
y_pred = prep.predict(X_test)

In [ ]:
print("R2 of", r2_score(y_test.TV, y_pred))
print("RMSE of", np.sqrt(mean_squared_error(y_test.TV, y_pred)))
print("Adjusted RMSE: ", np.sqrt(mean_squared_error(y_test.TV, y_pred))*14.57/74.86)

# Appendix


---



In this part of the report we introduice some of the ideas we had to solve the problem that were not as successful as expected.

## Log Transformation

Since there was a lot of extreme values in the Washing_machine and Kettle target, we decided to try to apply a log transformation on the values: 
$$ y_{new} = log(1+y) $$

Let's see the results in the case of washing_machine:

In [ ]:
#Loading data
X_train = Xtrain.copy()
y_train = ytrain.copy()

#Applying the log transformation
y_new = np.log(1+y_train.washing_machine)

#Plot
fig, ax = plt.subplots(1,2, figsize=(18,3))
ax[0].hist(y_new)
ax[0].set_title("Histogram of all the values")
ax[1].hist(y_new[y_new>=1])
ax[1].set_title("Histogram for all values >= 1")

The data seems more normally distributed when values are higher than 1. This could help our model. 

This the data preparation and the model that will be used to predict $y_{new}$.

In [ ]:
class washing_machine():
    
    def __init__(self):
        self.model = 0
        
    def shift_data(self, data, lag=30):
        series = [data.shift(i) for i in range(lag-1, -1, -1)]
        new_df = pd.concat(series, axis=1)
        name_col = []
        
        for i in range(lag, 0, -1):
            var = "consumption -" + str(i)
            name_col.append(var)
        
        new_df.columns = name_col
        new_df["std"] = new_df.std(axis=1)
        new_df["mean"] = new_df.mean(axis=1)
        
        return new_df
    
    def transform(self, X_train):
        
        #Environment information
        X_train = X_train.drop(columns="Unnamed: 9")
        X_train.time_step = pd.to_datetime(X_train.time_step)
        X_train["week"] = X_train["time_step"].map(lambda x: x.week)
        X_train["weekday"] = X_train["time_step"].map(lambda x: x.weekday)
        X_train["hour"] = X_train["time_step"].map(lambda x: x.hour)
        
        #Missing values
        X_train.fillna(method="ffill", inplace=True) #Get the previous non null values
        
        #Supervised consumption
        supervised_consumption = self.shift_data(X_train.consumption)
        X_train = X_train.join(supervised_consumption)
        
        #One hot encoding
        X_train_hour = pd.Categorical(X_train.hour, categories = [i for i in range(24)])
        X_train_weekday = pd.Categorical(X_train.weekday, categories = [i for i in range(7)])
        X_train_week = pd.Categorical(X_train.week, categories = [i for i in range(52)])
        
        X_train = X_train.join(pd.get_dummies(X_train_hour, prefix="hour"))
        X_train = X_train.join(pd.get_dummies(X_train_weekday, prefix="weekday"))
        X_train = X_train.join(pd.get_dummies(X_train_week, prefix="week"))
        
        X_train.drop(columns=["time_step", "hour", "weekday", "week"], inplace=True)
        
        return X_train
    
    def fit(self, X_train, y_train):
        lgb_train = lgb.Dataset(X_train.values, y_train)
        params = {
            'boosting_type': 'gbdt',
            'objective': 'regression',
            'metric': {'l2', 'l1'},
            'num_leaves': 50,
            'learning_rate': 0.05,
            'feature_fraction': 0.9,
            'bagging_fraction': 0.8,
            'bagging_freq': 5,
            'verbose': 0}

        self.model = lgb.train(params, lgb_train,
                             num_boost_round=5000)
        
    def predict(self, X_test):
        y_pred = self.model.predict(X_test, num_iteration=self.model.best_iteration)
        y_pred = np.where(y_pred<0,0,y_pred)
        return y_pred

In [ ]:
wash = washing_machine()
X_train = wash.transform(X_train)

#Drop na
data = X_train.join(y_new)
data = data.dropna()
X_train = data.iloc[:,:-1]
y_train = data.iloc[:,-1:]

#Train-test splot
X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, 
                                                    shuffle=False, random_state=42)

#Fit and predict
wash.fit(X_train, y_train)
y_pred = np.exp(wash.predict(X_test)) - 1

print("R2 of ", r2_score(np.exp(y_test)-1, y_pred))
print("RMSE of ", np.sqrt(mean_squared_error(np.exp(y_test)-1, y_pred)))

On the testing set the scores are quite bad, so we decided to drop that idea.

## Model Stacking

We also tried to do some model stacking for Kettle. The models used are the following: \\
- Random Forest \\
- XGBoost \\
- LGBM 

Then, we took them as input of a Neural Network with the following structure: \\
- First layer: 12 neurons \\
- Second layer: 6 neurons \\
- Third layer: 3 neurons \\
- Output layer (1 neuron)

To fit the model, we split our training data into 4 sets of equal size. \\
- The 1st set was used to train the Random Forest model \\
- The 2nd set was used to train the XGBoost model \\
- The 3rd set was used to train the LGBM model \\
- The 4th set was used to train the neural network 

Let's see the application of that in Python. 

In [ ]:
def shift_data(data, lag=20):
    series = [data.shift(i) for i in range(lag-1, -1, -1)]
    new_df = pd.concat(series, axis=1)
    name_col = []

    for i in range(lag, 0, -1):
        var = "consumption -" + str(i)
        name_col.append(var)

    new_df.columns = name_col
    new_df["std"] = new_df.std(axis=1)
    new_df["mean"] = new_df.mean(axis=1)

    return new_df

def transform(X_train):
    X_train = X_train[["time_step", "consumption"]]
    X_train.time_step = pd.to_datetime(X_train.time_step)

    X_train["hour"] = X_train["time_step"].map(lambda x: x.hour)
    X_train["weekday"] = X_train["time_step"].map(lambda x: x.dayofweek)
    X_train["week"] = X_train["time_step"].map(lambda x: x.week)

    #Missing values
    X_train.fillna(method="ffill", inplace=True) #Get the previous non null values

    #Supervised consumption
    supervised_consumption = shift_data(X_train.consumption)

    #One hot encoding
    X_train_hour = pd.Categorical(X_train.hour, categories = [i for i in range(24)])
    X_train_weekday = pd.Categorical(X_train.weekday, categories = [i for i in range(7)])
    X_train_week = pd.Categorical(X_train.week, categories = [i for i in range(52)])

    X_train = X_train.join(pd.get_dummies(X_train_hour, prefix="hour"))
    X_train = X_train.join(pd.get_dummies(X_train_weekday, prefix="weekday"))
    X_train = X_train.join(pd.get_dummies(X_train_week, prefix="week"))

    X_train.drop(columns=["time_step","hour", "weekday", "week"], inplace=True)

    return X_train.join(supervised_consumption)

In [ ]:
#Loading data
X_train = Xtrain.copy()
y_train = ytrain.copy()
X_train = transform(X_train)

#Drop na for training
data = X_train.join(y_train)
data = data.dropna()
X_train = data.iloc[:,:-5]
y_train = data.iloc[:,-5:]
X_train.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)

#Split the data into 4 sets
index = X_train.index
new_index = np.random.permutation(index)[:-5]
size = int(np.round(len(new_index)/4))
a = [new_index[i:i + size] for i in range(0, len(new_index), size)]

X_train0 = X_train.iloc[a[0],:]
X_train1 = X_train.iloc[a[1],:]
X_train2 = X_train.iloc[a[2],:]
X_train3 = X_train.iloc[a[3],:]

y_train0 = y_train.iloc[a[0],:]
y_train1 = y_train.iloc[a[1],:]
y_train2 = y_train.iloc[a[2],:]
y_train3 = y_train.iloc[a[3],:]

# Random Forest
from sklearn.ensemble import RandomForestRegressor
model0 = RandomForestRegressor()
model0.fit(X_train0, y_train0.kettle)
feature0 = model0.predict(X_train3)

# XGB
model1 = xgb.XGBRegressor()
model1.fit(X_train1, y_train1.kettle)
feature1 = model1.predict(X_train3)

#LGBM
model2 = lgb.LGBMRegressor()
model2.fit(X_train2, y_train2.kettle)
feature2 = model2.predict(X_train3)

# Input for the Neural Network
new_df = pd.DataFrame({"RF": feature0, "XGB": feature1, "LGB": feature2})
new_df = np.where(new_df<0, 0, new_df)

# Neural Network structure
from tensorflow.python.keras.layers import Dense, LSTM
from tensorflow.python.keras import Sequential

model = Sequential()
model.add(Dense(12, input_dim= new_df.shape[1], activation="relu"))
model.add(Dense(6, activation="relu"))
model.add(Dense(3, activation="relu"))
model.add(Dense(1))

model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(new_df, y_train3.kettle, epochs=10, batch_size=10)

In [ ]:
y_pred = model.predict(new_df)
y_pred = np.where(y_pred<0, 0, y_pred)

print("R2 of ", r2_score(y_train3.kettle, y_pred))
print("RMSE of ", np.sqrt(mean_squared_error(y_train3.kettle, y_pred)))

The results are quite good but it was not our best result on the online platform.

## Other ideas for Kettle

We had the following ideas for Kettle:

**Feature Label**\
Adding a feature that labels 1 the rows where kettle is different than zero and 0 the rows where kettle is null. We created this feature by using an XGBoost Classifier. As the dataset is very unbalanced (there is a majority of rows labeled 0), we used the parameter scale_pos_weight=3.7 to Control the balance of positive and negative weights. 

**Additionnal features** \
Adding the temporal features half_hour, season, timeofday, daylight inversed.

**Validation set**\
We noticed that the test set is indexed by the months January to June 2014. In the training set we only have data from March 17 to the end of December 2013. So to take this into account, we chose a validation set with a month that is not present in the training set (month 7) and 4 four weeks of months that are present in the training set (week 14 of month 3, week 15 of month 4, week 43 and 44 of October).


In [ ]:
class kettle():
    
    def __init__(self):
        self.model = 0
        
    def label(self,x):
        if x == 0 :
            return 0
        else :
            return 1
        
    def prep_kettle(self,X_train,y_train):
        data = X_train
        data_month = data['month'].copy()
        data_week = data['week'].copy()

        #past consumption
        for i in range(10, -1, -1):
            data['conso-'+str(i)] = data.consumption.shift(i)

        #one hot encoding
        data = data.join(pd.get_dummies(data.half_hour, prefix="half_hour"))
        data = data.join(pd.get_dummies(data.weekday, prefix="weekday"))
        data = data.join(pd.get_dummies(data.week, prefix="week"))
        data = data.join(pd.get_dummies(data.season, prefix="season"))
        data = data.join(pd.get_dummies(data.timeofday, prefix="timeofday"))

        data = data.drop(columns=["half_hour","weekday","week","season","timeofday","hour","month",'conso-0'])
        data = data.drop(columns=["visibility","humidity","windchill","wind","pressure"])

        return data, data_month, data_week
    
    def XGB(self,X_train,y_train,k):
        data_kettle, data_month, data_week = self.prep_kettle(X_train,y_train)
        data_kettle = pd.concat([data_kettle,y_train.kettle], axis=1)
        data_kettle['label'] = data_kettle.kettle.apply(lambda x: self.label(x))

        #Train-test split
        test_set = pd.concat([data_kettle[data_month == 7],
                          data_kettle[data_week == {14,15,43,44}]])

        train_set = data_kettle.loc[np.delete(data_kettle.index, test_set.index)]

        X_train = train_set.drop(['kettle','label'], axis = 1)
        y_train = train_set.label

        X_test = test_set.drop(['kettle','label'], axis = 1)
        y_test = test_set.label

        #Model
        model = XGBClassifier(scale_pos_weight=k,random_state=42)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        return y_pred
    
    def transform(self, X_train,y_train):       
        X_train = X_prep(X_train)
        y_train = y_prep(y_train)
        data_kettle, data_month, data_week = self.prep_kettle(X_train,y_train)
        y_test_label = self.XGB(X_train,y_train,3.7)

        #Splitting train and test sets
        X_test = pd.concat([data_kettle[data_month == 7],
                            data_kettle[data_week == {14,15,43,44}]])
        X_train = data_kettle.loc[np.delete(data_kettle.index, X_test.index)]
        
        y_test = pd.concat([y_train.kettle[data_month == 7],
                            y_train.kettle[data_week == {14,15,43,44}]])
        y_train = y_train.kettle.loc[np.delete(y_train.kettle.index, y_test.index)]
        
        #Adding the feature label
        X_train['label'] = y_train.apply(lambda x: self.label(x))
        X_train = X_train.join(pd.get_dummies(X_train.label, prefix="label"))
        X_train = X_train.drop('label', axis = 1)
        
        X_test['label'] = y_test_label
        X_test = X_test.join(pd.get_dummies(X_test.label, prefix="label"))
        X_test = X_test.drop('label', axis = 1)
        
        #Ensuring the same features for X_train and X_test
        col_dif = set(X_train.columns) - set(X_test.columns)
        for i in col_dif:
            X_test[i] = 0

        col_dif = set(X_test.columns) - set(X_train.columns)
        for i in col_dif:
            X_train[i] = 0
            
        return X_train, y_train, X_test, y_test
    
    def fit_predict(self, X_train, y_train,X_test):
        model_kettle = xgb.XGBRegressor(random_state=42) 
        model_kettle.fit(X_train.values, y_train.values)
        y_pred_kettle = model_kettle.predict(X_test.values)
        y_pred_kettle[y_pred_kettle<0] = 0
        return y_pred_kettle

In [ ]:
#Loading data
X_train = Xtrain.copy()
y_train = ytrain.copy()

#Transform
kettle = kettle()
X_train, y_train, X_test, y_test = kettle.transform(X_train,y_train)

#Model
y_pred = kettle.fit_predict(X_train, y_train,X_test)

In [ ]:
print("R2 of", r2_score(y_test, y_pred))
print("RMSE of", np.sqrt(mean_squared_error(y_test, y_pred)))
print("Adjusted RMSE: ", np.sqrt(mean_squared_error(y_test, y_pred))*4.95/74.86)

This is a satisfying RMSE for kettle, however when we submitted on the platform using this method, we obtained a score worse than before and decided to drop this idea.